# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_LDA(lattice, atoms, positions; symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770150776458                   -0.53    9.0         
  2   -2.771682519418       -2.81       -1.29    1.0    134ms
  3   -2.771714161954       -4.50       -2.67    1.0    160ms
  4   -2.771714713827       -6.26       -4.04    2.0    178ms
  5   -2.771714715212       -8.86       -4.65    2.0    155ms
  6   -2.771714715248      -10.45       -5.81    1.0    150ms
  7   -2.771714715250      -11.69       -6.20    2.0    160ms
  8   -2.771714715250      -13.23       -6.91    1.0    146ms
  9   -2.771714715250      -14.57       -7.87    1.0    172ms
 10   -2.771714715250   +  -13.76       -8.19    2.0    170ms


-0.00013457390223196094

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_LDA(lattice, atoms, positions;
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770509143287                   -0.52    9.0         
  2   -2.771779617548       -2.90       -1.32    1.0    135ms
  3   -2.771801626357       -4.66       -2.44    1.0    138ms
  4   -2.771802007125       -6.42       -3.15    1.0    168ms
  5   -2.771802074348       -7.17       -4.35    2.0    158ms
  6   -2.771802074443      -10.02       -4.66    1.0    164ms
  7   -2.771802074474      -10.51       -5.55    1.0    143ms
  8   -2.771802074476      -11.75       -6.03    2.0    164ms
  9   -2.771802074476      -12.97       -7.04    1.0    163ms
 10   -2.771802074476      -13.44       -7.45    2.0    174ms
 11   -2.771802074476   +  -13.81       -7.98    1.0    168ms
 12   -2.771802074476   +  -14.24       -8.95    2.0    173ms


0.01761222129248772

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457390223196094
Displaced dipole:  0.01761222129248772
Polarizability :   1.774679519471968


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920941369e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766553405811e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852759355855e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694608871268e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088787947112e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.307215050904e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 1.536211545781e-12
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 1.536211545781e-12
[ Info: GMRES linsolve in iter 2; step 1: normres = 8.946879360370e-10
[ Info: GMRES linsolve in iter 2; step 2: normres = 8.679588858638e-11
[ Info: GMRES linsolve in iter 2; step 3: normres = 1.507473299673e-12
[ Info: GMRES linsolve in iter 2; finished at step 3: normres = 1.507473299673e-12
┌ Info: GMRES linsolve converged at iteration 2, step 3:
│ *  norm of residual = 1.507467348135961e-12
└ *  number of operat

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.